In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
import os
import random
from keras.api.applications import InceptionV3
import cv2
from keras.api.layers import StringLookup
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
import cv2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
dataset_path = '../datasets/tp5'

In [4]:
train = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
test = pd.read_csv(os.path.join(dataset_path, 'test.csv'))


In [5]:
for vid in train['video_name']:
    size = os.path.getsize(os.path.join(dataset_path, 'train', vid))
    print(f'video : {vid}, size : {size/1000} kb')

video : v_CricketShot_g08_c01.avi, size : 292.132 kb
video : v_CricketShot_g08_c02.avi, size : 294.336 kb
video : v_CricketShot_g08_c03.avi, size : 205.096 kb
video : v_CricketShot_g08_c04.avi, size : 261.03 kb
video : v_CricketShot_g08_c05.avi, size : 389.568 kb
video : v_CricketShot_g08_c06.avi, size : 321.134 kb
video : v_CricketShot_g08_c07.avi, size : 372.5 kb
video : v_CricketShot_g09_c01.avi, size : 287.62 kb
video : v_CricketShot_g09_c02.avi, size : 263.436 kb
video : v_CricketShot_g09_c03.avi, size : 320.254 kb
video : v_CricketShot_g09_c04.avi, size : 267.27 kb
video : v_CricketShot_g09_c05.avi, size : 302.182 kb
video : v_CricketShot_g09_c06.avi, size : 363.102 kb
video : v_CricketShot_g09_c07.avi, size : 210.084 kb
video : v_CricketShot_g10_c01.avi, size : 470.474 kb
video : v_CricketShot_g10_c02.avi, size : 355.276 kb
video : v_CricketShot_g10_c03.avi, size : 304.658 kb
video : v_CricketShot_g10_c04.avi, size : 205.012 kb
video : v_CricketShot_g10_c05.avi, size : 325.316 k

In [ ]:
for vid in test['video_name']:
    size = os.path.getsize(os.path.join(dataset_path, 'test', vid))
    print(f'video : {vid}, size : {size/1000} kb')

video : v_CricketShot_g01_c01.avi, size : 265.132 kb
video : v_CricketShot_g01_c02.avi, size : 356.072 kb
video : v_CricketShot_g01_c03.avi, size : 369.472 kb
video : v_CricketShot_g01_c04.avi, size : 275.182 kb
video : v_CricketShot_g01_c05.avi, size : 369.376 kb
video : v_CricketShot_g01_c06.avi, size : 285.596 kb
video : v_CricketShot_g01_c07.avi, size : 266.214 kb
video : v_CricketShot_g02_c01.avi, size : 282.654 kb
video : v_CricketShot_g02_c02.avi, size : 294.998 kb
video : v_CricketShot_g02_c03.avi, size : 335.092 kb
video : v_CricketShot_g02_c04.avi, size : 278.102 kb
video : v_CricketShot_g02_c05.avi, size : 456.29 kb
video : v_CricketShot_g02_c06.avi, size : 338.534 kb
video : v_CricketShot_g02_c07.avi, size : 319.17 kb
video : v_CricketShot_g03_c01.avi, size : 374.054 kb
video : v_CricketShot_g03_c02.avi, size : 431.122 kb
video : v_CricketShot_g03_c03.avi, size : 383.668 kb
video : v_CricketShot_g03_c04.avi, size : 419.844 kb
video : v_CricketShot_g03_c05.avi, size : 428.03

In [7]:
print(train.head()) # Print the 5 first line of the df 
print('---------------------')
print(train.describe()) # Print some stats on our df
print('------------------------------')
print(train.info()) # Print a summary of our df 

                  video_name          tag
0  v_CricketShot_g08_c01.avi  CricketShot
1  v_CricketShot_g08_c02.avi  CricketShot
2  v_CricketShot_g08_c03.avi  CricketShot
3  v_CricketShot_g08_c04.avi  CricketShot
4  v_CricketShot_g08_c05.avi  CricketShot
---------------------
                       video_name    tag
count                         594    594
unique                        594      5
top     v_CricketShot_g08_c01.avi  Punch
freq                            1    121
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594 entries, 0 to 593
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   video_name  594 non-null    object
 1   tag         594 non-null    object
dtypes: object(2)
memory usage: 9.4+ KB
None


In [8]:
targets = train['tag'].unique()
print(targets)

['CricketShot' 'PlayingCello' 'Punch' 'ShavingBeard' 'TennisSwing']


In [9]:
def feature_extractor():
    model = InceptionV3(
                include_top=False,
                weights="imagenet",
                input_tensor=None,
                input_shape=None,
                pooling='avg',
                name="inception_v3",
            )
    model.trainable = False
    return model

In [10]:
layer = StringLookup(vocabulary=targets, num_oov_indices=0, output_mode='one_hot')

In [11]:
train_y = tf.reshape(layer(train['tag']), shape=(-1,5))
test_y = tf.reshape(layer(test['tag']), shape=(-1, 5))

In [12]:
print(train_y)
print('-------------------')
print(test_y)

tf.Tensor(
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]], shape=(594, 5), dtype=int64)
-------------------
tf.Tensor(
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]], shape=(224, 5), dtype=int64)


In [13]:
def crop_center_square(frame):
    y, x = frame.shape[:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    
    return frame[start_y:start_y + min_dim, start_x:start_x + min_dim]

In [14]:
def load_video(video_path, fps, shape=(224,224)):
    count = 0
    frame_count = 0
    cap = cv2.VideoCapture(video_path)
    nb_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    step = nb_frames // fps
    
    frames = []
    for frame_count in range(fps):
        
        frame_index = frame_count * step
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        
        ret, frame = cap.read()


        frame = crop_center_square(frame)
        frame = cv2.resize(frame, shape)

        frames.append(frame)
    
    cap.release()
    return np.array(frames) / 255.0

In [17]:
def process_video(video_path, fps=25):
    frames = load_video(video_path, fps)
    frames_features = np.empty(shape=(fps, 1, 2048))
    frame_mask = np.ones(shape=(fps))
    fe = feature_extractor()
    
    for i, frame in enumerate(frames):
        frame = np.expand_dims(frame, axis=0)
        features = fe(frame)
        frames_features[i] = features
        if frame.all():
            frame_mask[i] = 0
        break
        
    return np.array(frames_features), np.array(frame_mask)

In [18]:
def prepare_all_video(path):
    
    videos_path = []
    for vid in os.listdir(path):
        videos_path.append(os.path.join(path, vid))

    print(videos_path)

    all_features = []
    all_masks = []

    with ThreadPoolExecutor(max_workers=8) as exec:
        results = list(tqdm(exec.map(process_video, videos_path), total=len(videos_path)))
    
    for features, mask in results:
        all_features.append(features)
        all_masks.append(mask)
    return np.array(all_features), np.array(all_masks)

In [19]:
train_path = '../datasets/tp5/train'
test_path = '../datasets/tp5/test'
train_features, train_masks = prepare_all_video(train_path)
test_features, test_masks = prepare_all_video(test_path)

['../datasets/tp5/train\\v_CricketShot_g08_c01.avi', '../datasets/tp5/train\\v_CricketShot_g08_c02.avi', '../datasets/tp5/train\\v_CricketShot_g08_c03.avi', '../datasets/tp5/train\\v_CricketShot_g08_c04.avi', '../datasets/tp5/train\\v_CricketShot_g08_c05.avi', '../datasets/tp5/train\\v_CricketShot_g08_c06.avi', '../datasets/tp5/train\\v_CricketShot_g08_c07.avi', '../datasets/tp5/train\\v_CricketShot_g09_c01.avi', '../datasets/tp5/train\\v_CricketShot_g09_c02.avi', '../datasets/tp5/train\\v_CricketShot_g09_c03.avi', '../datasets/tp5/train\\v_CricketShot_g09_c04.avi', '../datasets/tp5/train\\v_CricketShot_g09_c05.avi', '../datasets/tp5/train\\v_CricketShot_g09_c06.avi', '../datasets/tp5/train\\v_CricketShot_g09_c07.avi', '../datasets/tp5/train\\v_CricketShot_g10_c01.avi', '../datasets/tp5/train\\v_CricketShot_g10_c02.avi', '../datasets/tp5/train\\v_CricketShot_g10_c03.avi', '../datasets/tp5/train\\v_CricketShot_g10_c04.avi', '../datasets/tp5/train\\v_CricketShot_g10_c05.avi', '../dataset

100%|██████████| 594/594 [16:46<00:00,  1.70s/it]


['../datasets/tp5/test\\v_CricketShot_g01_c01.avi', '../datasets/tp5/test\\v_CricketShot_g01_c02.avi', '../datasets/tp5/test\\v_CricketShot_g01_c03.avi', '../datasets/tp5/test\\v_CricketShot_g01_c04.avi', '../datasets/tp5/test\\v_CricketShot_g01_c05.avi', '../datasets/tp5/test\\v_CricketShot_g01_c06.avi', '../datasets/tp5/test\\v_CricketShot_g01_c07.avi', '../datasets/tp5/test\\v_CricketShot_g02_c01.avi', '../datasets/tp5/test\\v_CricketShot_g02_c02.avi', '../datasets/tp5/test\\v_CricketShot_g02_c03.avi', '../datasets/tp5/test\\v_CricketShot_g02_c04.avi', '../datasets/tp5/test\\v_CricketShot_g02_c05.avi', '../datasets/tp5/test\\v_CricketShot_g02_c06.avi', '../datasets/tp5/test\\v_CricketShot_g02_c07.avi', '../datasets/tp5/test\\v_CricketShot_g03_c01.avi', '../datasets/tp5/test\\v_CricketShot_g03_c02.avi', '../datasets/tp5/test\\v_CricketShot_g03_c03.avi', '../datasets/tp5/test\\v_CricketShot_g03_c04.avi', '../datasets/tp5/test\\v_CricketShot_g03_c05.avi', '../datasets/tp5/test\\v_Crick

100%|██████████| 224/224 [06:18<00:00,  1.69s/it]


In [20]:
print(f'train_ft : {train_features.shape}, train_msk : {train_masks.shape}')
print(f'test_ft : {test_features.shape}, test_msk : {test_masks.shape}')

train_ft : (594, 25, 1, 2048), train_msk : (594, 25)
test_ft : (224, 25, 1, 2048), test_msk : (224, 25)


In [31]:
from keras.api.layers import Input, TimeDistributed, LSTM, Dropout, Dense, Masking, Reshape
from keras.api.models import Model
from keras.api.optimizers import Adam
from keras.api.callbacks import ModelCheckpoint

In [53]:
def sequence_model():
    
    features_input = Input(shape=(None, 25, 1, 2048), name="features_input")
    reshaped_features = Reshape((-1, 25, 2048))(features_input)
    masks_input = Input(shape=(None, 25), name="masks_input")

    x = TimeDistributed(Dense(512, activation='relu'))(masks_input)
    x = TimeDistributed(Dense(256, activation='relu'))(x)

    x = LSTM(512, return_sequences=True)(reshaped_features)
    x = LSTM(256, return_sequences=True)(x)
    x = LSTM(128)(x)

    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(len(layer.get_vocabulary()), activation='softmax')(x)

    model = Model(inputs=[features_input, masks_input], outputs=output)

    opt = Adam(learning_rate=0.0005)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [54]:
def run_experiment(train_features, train_masks, train_label, test_features, test_masks, test_labels):
    model = sequence_model()
    checkpoint = ModelCheckpoint('./models_checkpoints/best_model.keras', save_best_only=True, monitor="val_accuracy", mode="max")
    print(model.summary())
    
    history = model.fit(
        x=[train_features, train_masks],
        y=train_label,
        epochs=20,
        callbacks=[checkpoint]
    )
    
    model.load_weights('./models_checkpoints/best_model.keras')
    evaluation = model.evaluate(
        x=[test_features, test_masks],
        y=test_labels
    )
    
    predictions = np.argmax(model.predict(x=[test_features, test_masks]), axis=1)
    print(classification_report(test_labels, predictions))
    
    print(f'Model evaluation : {evaluation}')
    return history

In [55]:
history = run_experiment(train_features, train_masks, train_y, test_features, test_masks, test_y)

ValueError: Input 0 of layer "lstm_20" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, None, 25, 2048)

c:\Users\Jalil\Desktop\Ecole\M2TSI\tpenv\lib\site-packages\keras\src\layers\layer.py:934: UserWarning: Layer 'reshape_4' (of type Reshape) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
